# data processing into a single dataframe


In [4]:
import pandas as pd
'''


for i in 2 3 4; do python3 keep_relevant_headers.py \
../data/chr2-4_with_gene/chr$i\_gene.tsv \
../data/chr2-4_with_gene/chr$i.out_set1.tsv \
--cols chr,pos,ref,alt,aapos,codonpos,Polyphen2_HVAR_pred,\
GenoCanyon_score,LINSIGHT,clinvar_clnsig,RVIS_ExAC,ExAC_cnv.score,SORVA_LOF_MAF0.005_HetOrHom; done

'''


df_all = pd.read_csv('../../../data/chr2-4_with_gene/chr2.out_set5.tsv', sep='\t', encoding = "UTF-8")
df_all = df_all.set_index(['chr','pos', 'ref', 'alt']) #index by identifier

for c in range (3,5):
    #print(c)
    path = '../../../data/chr2-4_with_gene/chr' + str(c) + '.out_set5.tsv'
    df = pd.read_csv(path, sep='\t', encoding = "UTF-8")
    df = df.set_index(['chr','pos', 'ref', 'alt']) #index by identifier
    df_all= df_all.append(df)
    
print('concat shape', df_all.shape)

df = df_all
print("shape before", df.shape)
#print(df.head)
df = df.replace({'.': 0})  ## FIXME: atm replace with nan and drop: later find a way to impute, maybe 0 but not sure
#print("SUM", pd.isnull(df).sum())  ## Use this to see how many elements have nan 
df = df.dropna()
print("shape after dropping all data with score = '.'", df.shape)
print(df.head())

## still contains some mixed labels, see them by: df[~df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]

### prior to remapping of clinical significance labels to only pathogenic/benign, check out the distirbutions:
#print("TOTAL SUM OF clinsig: \n", df['clinvar_clnsig'].value_counts())
#print("PERCENT OF clinsig: \n", df['clinvar_clnsig'].value_counts(normalize=True))
#print("CLINVAR LABELS \n", df.groupby('clinvar_clnsig').count())

## Map labels to 1/0 and remove Unkn
clinvar_labels={
    'Pathogenic': 1,
    'Likely_pathogenic': 1,
    'Pathogenic/Likely_pathogenic': 1,
    'Benign/Likely_benign': 2,
    'Likely_benign': 2,
    'Benign': 2,
    'Uncertain_significance': 0
}

features_mapping = {
    'Polyphen2_HVAR_pred': {
        'D': -2,
        'P': -1, 
        'B': 1
    }
}

df = df.replace(features_mapping)

df = df[df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]  # best to drop everything that isn't in the dict
df['Problematic'] = df['clinvar_clnsig'].map(clinvar_labels)

df= df.drop(columns=['clinvar_clnsig'])#, 'clinvar_id'])  #tbh, I don't know why I kept them. consider adding to index
print("shape after dropping mixed labels", df.shape)

### pickle to reuse dataframe:
df.to_pickle("../pickled/chr2-4_unk_featureset5.pkl")

#print("PROBLEMATIC LABEL DISTRIBUTION \n", df.groupby('Problematic').count())
print("TOTAL SUM OF LABELS: \n", df['Problematic'].value_counts())
print("PERCENT OF LABELS: \n", df['Problematic'].value_counts(normalize=True))

concat shape (44399, 9)
shape before (44399, 9)
shape after dropping all data with score = '.' (44390, 9)
                     aapos codonpos Polyphen2_HVAR_pred     GenoCanyon_score  \
chr pos     ref alt                                                            
2   272223  G   A      102        1                   0    0.999999999999996   
                A       50        2                   0    0.999999999999996   
    277003  A   G      106        2                   B      0.9777236555582   
    1436306 C   A      135        2                   D   0.0141758883741757   
    1436345 C   T      148        2                   D  4.51595965165985E-4   

                    LINSIGHT          clinvar_clnsig     RVIS_ExAC  \
chr pos     ref alt                                                  
2   272223  G   A          0  Uncertain_significance  -0.080196141   
                A          0  Uncertain_significance  -0.080196141   
    277003  A   G          0                  Benign 